In [1]:
from flask import Flask, render_template, request, jsonify
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os
import speech_recognition as sr
import io


In [2]:

sentiment_labels = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]

In [3]:
saved_model_dir = r"D:\Sentimental analysis\bert_sentiment_model"
tokenizer = AutoTokenizer.from_pretrained(saved_model_dir)
model = AutoModelForSequenceClassification.from_pretrained(saved_model_dir)


In [4]:
print(model)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [5]:
def convert_mp3_to_text(audio_file):
    try:
        # Initialize the recognizer
        recognizer = sr.Recognizer()
        
        # Load the audio file
        with sr.AudioFile(audio_file) as source:
            # Listen for the data from the audio file
            audio_data = recognizer.record(source)
            
            # Convert speech to text
            text = recognizer.recognize_google(audio_data)
            
            return text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract predicted sentiment
    predicted_scores = torch.softmax(outputs.logits, dim=1)
    predicted_label = sentiment_labels[predicted_scores.argmax().item()]

    return predicted_label


In [6]:
text=convert_mp3_to_text(r"D:\Sentimental analysis\inputs_wav_form\review_1.wav")

In [7]:
print(text)

the food was amazing and the service was excellent


In [8]:
analyze_sentiment(text)

'Very Positive'